In [3]:
import glob
import cv2 as cv
import cv2
import numpy as np
import numpy
import matplotlib.pyplot as plt
import skimage.color
import skimage.filters
from numpy import asarray
import math
from skimage.measure import regionprops_table, label
import pandas as pd
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split, GridSearchCV

In [4]:
def contouring(image):
    
    
    contours, hierarchy = cv2.findContours(image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    biggest_contour = max(contours, key=cv2.contourArea)
 
    mask = np.zeros(image.shape, np.uint8)
    
    cv2.drawContours(mask, [biggest_contour], -1, 255, -1)
    
    return mask

In [7]:
temp_img = []
output_img = []

out = glob.glob("output/baso/*.bmp")
out2 = glob.glob("output/eosi/*.bmp")
out3 = glob.glob("output/lymp/*.bmp")
out4 = glob.glob("output/mixt/*.bmp")
out5 = glob.glob("output/mono/*.bmp")
out6 = glob.glob("output/neut/*.bmp")

outs = [out,out2,out3,out4,out5,out6]

for k in outs:
    for file in k:
        img2 = cv2.imread(file)
        to_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        output_img.append(to_gray)


path = glob.glob("baso/*.bmp")
path2 = glob.glob("eosi/*.bmp")
path3 = glob.glob("lymp/*.bmp")
path4 = glob.glob("mixt/*.bmp")
path5 = glob.glob("mono/*.bmp")
path6 = glob.glob("neut/*.bmp")

paths = [path,path2,path3,path4,path5,path6]

for j in paths:
    for file in j:
        img = cv2.imread(file)
        bright = skimage.exposure.rescale_intensity(img.astype(np.uint8),in_range=(1,40), out_range=(0,200))

        # converting to LAB color space
        lab= cv2.cvtColor(bright.astype(np.uint8), cv2.COLOR_BGR2LAB)
        l_channel, a, b = cv2.split(lab)

        # Applying CLAHE to L-channel
        clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8,8))
        cl = clahe.apply(l_channel)

        # # merge the CLAHE enhanced L-channel with the a and b channel
        limg = cv2.merge((cl,a,b))

        # Converting image from LAB Color model to BGR color spcae
        enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
        
        hsv = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2HSV)
        lower = [90, 60, 100]
        upper = [200, 200, 255]
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        
        fixed = cv.medianBlur(hsv, 3)
        
        mask = cv2.inRange(fixed, lower, upper)
        masked_img = contouring(mask)
        temp_img.append(masked_img)

#         plt.imshow(masked_img, cmap='gray')
#         plt.show()

iou_scores = []    

for i in range(0,len(output_img)):
    intersection = np.logical_and(temp_img[i],output_img[i])
    union = np.logical_or(temp_img[i],output_img[i])
    iou_score = np.sum(intersection)/np.sum(union)
    iou_scores.append(iou_score)


total = 0
for i in range(len(iou_scores)):
    total = total + iou_scores[i]

avg = total / len(iou_scores)

print("IoU =", avg * 100)

IoU = 72.19025653498001


In [8]:
def fourier_descriptor(x, y):
    complex_ = np.array([complex(i, j) for i, j in zip(x, y)])
    F = np.fft.fft(complex_)

    F_positive = abs(F)

    scale = abs(F[1])
    F_trunc = F_positive[1:11]
    F_final = F_trunc / scale
    return F_final

In [13]:
df = pd.DataFrame()
circularity = []
temp_x = []
temp_y = []
decriptor = []
for i in range (0,250):
    bw = temp_img[i] > 28
    label_img = label(bw)

    props_dict = regionprops_table(label_img, temp_img[i],properties=['area','perimeter', 'eccentricity','moments_hu'])

    features_table = [[] for label in props_dict['area']]
    for prop, values in props_dict.items():
        for i, value in enumerate(values):
            features_table[i].append(value)
    df = pd.concat([df,pd.DataFrame(props_dict)],ignore_index=True)

for i in range (0,250):
    circularity.append((4* math.pi * (df.loc[i,"area"]) / math.pow(df.loc[i,"perimeter"],2)))
    # descriptor   
    # contours
    contours, hierarchy = cv2.findContours(temp_img[i], cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    temp_x.append(contours[0][:, 0][:, 0])
    temp_y.append(contours[0][:, 0][:, 1])
    
    decriptor.append(fourier_descriptor(temp_x[i], temp_y[i]))


In [14]:
df["Class"] = ""

#WBC baso
for i in range (0,53):   
    df.loc[i,"Class"] = "Baso"
#WBC eosi
for i in range (53,92):   
    df.loc[i,"Class"] = "Eosi"
#WBC lymp
for i in range (92,144):   
    df.loc[i,"Class"] = "Lymp"        
#WBC mixt
for i in range (144,152):   
    df.loc[i,"Class"] = "Mixt"   
    
#WBC mono
for i in range (152,200):   
    df.loc[i,"Class"] = "Mono"
    
#WBC neut
for i in range (200,250):   
    df.loc[i,"Class"] = "Neut"    
    

# df.insert(4,"circularity",circularity,True)
# df.insert(5,"descriptor",decriptor,True)
df

,area,perimeter,eccentricity,moments_hu-0,moments_hu-1,moments_hu-2,moments_hu-3,moments_hu-4,moments_hu-5,moments_hu-6,Class
0,5205,337.427453,0.623092,0.170066,0.001678,0.000244,3.136936e-06,7.157075e-11,1.251103e-07,4.900867e-11,Baso
1,3575,273.385822,0.466201,0.161714,0.000389,0.000046,9.556313e-08,-1.366224e-13,-6.287771e-10,1.455648e-13,Baso
2,3185,282.764502,0.421256,0.162676,0.000251,0.000049,1.289125e-06,1.649468e-12,-2.520397e-10,-1.010995e-11,Baso
3,2214,307.391919,0.580772,0.186488,0.001431,0.000091,3.563608e-06,-4.303796e-11,4.160451e-08,-4.765260e-11,Baso
4,4325,322.877200,0.637239,0.168882,0.001851,0.000050,2.332229e-06,-1.810399e-11,-8.681532e-08,1.753881e-11,Baso
...,...,...,...,...,...,...,...,...,...,...,...
245,2003,238.350288,0.822030,0.209906,0.011472,0.000741,1.720954e-04,5.833860e-08,1.743354e-05,-1.936786e-08,Neut
246,1717,220.208153,0.747404,0.210539,0.006658,0.002860,4.030499e-05,1.354209e-08,3.159443e-06,1.968891e-09,Neut
247,1983,346.190909,0.622777,0.248435,0.003572,0.002600,1.455052e-04,6.837768e-08,7.385881e-06,-5.775313e-08,Neut
248,1358,208.308658,0.853159,0.285309,0.026650,0.004777,8.662622e-04,-3.305087e-07,-3.126526e-05,-1.730838e-06,Neut


In [28]:
X = df.drop(columns = ['Class'])
y = df['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=2)
clf = svm.SVC(kernel="linear")
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print("Accuracy: {}%".format(clf.score(X_test, y_test) * 100))

Accuracy: 52.0%


In [12]:
import mlxtend
from mlxtend.evaluate import accuracy_score

std_acc = accuracy_score(y_test, predictions)
bin_acc = accuracy_score(y_test, predictions, method='binary', pos_label="Baso")
bin_acc2 = accuracy_score(y_test, predictions, method='binary', pos_label="Eosi")
bin_acc3 = accuracy_score(y_test, predictions, method='binary', pos_label="Lymp")
bin_acc4 = accuracy_score(y_test, predictions, method='binary', pos_label="Mixt")
bin_acc5 = accuracy_score(y_test, predictions, method='binary', pos_label="Mono")
bin_acc6 = accuracy_score(y_test, predictions, method='binary', pos_label="Neut")

print(f'Standard accuracy: {std_acc*100:.2f}%')
print(f'Class Baso accuracy: {bin_acc*100:.2f}%')
print(f'Class Eosi accuracy: {bin_acc2*100:.2f}%')
print(f'Class Lymp accuracy: {bin_acc3*100:.2f}%')
print(f'Class Mixt accuracy: {bin_acc4*100:.2f}%')
print(f'Class Mono accuracy: {bin_acc5*100:.2f}%')
print(f'Class Neut accuracy: {bin_acc6*100:.2f}%')

Standard accuracy: 64.00%
Class Baso accuracy: 80.00%
Class Eosi accuracy: 82.00%
Class Lymp accuracy: 98.00%
Class Mixt accuracy: 94.00%
Class Mono accuracy: 86.00%
Class Neut accuracy: 88.00%
